In [1]:
# Import des différents package
import os.path as op
import numpy as np
import re
import math

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from nltk.stem import SnowballStemmer
from nltk import pos_tag
from nltk.tokenize import word_tokenize

In [2]:
###############################################################################
# Load data
###############################################################################

print("Loading dataset")

from glob import glob
filenames_neg = sorted(glob(op.join('data', 'imdb1', 'neg', '*.txt')))
filenames_pos = sorted(glob(op.join('data', 'imdb1', 'pos', '*.txt')))

texts_neg = [open(f).read() for f in filenames_neg]
texts_pos = [open(f).read() for f in filenames_pos]
texts = texts_neg + texts_pos
y = np.ones(len(texts), dtype=np.int)
y[:len(texts_neg)] = 0.

print("%d documents" % len(texts))

Loading dataset
2000 documents


In [3]:
###############################################################################
# Question 1
###############################################################################

# Fonction qui renvoie le dictionnaire et la matrice
# qui indique pour chaque mot le nombre d'occurences
# dans chaque texte
# Pour anticiper les questions suivantes, des paramètre facultatifs
# ont été ajoutés :
#   - pour ne pas prendre en compte certains mots
#   - pour utiliser les racines des mots
def count_words(texts, english_stop=[], steeming=False):
    words = set()

    # On ne récupére les mots qui sont des "vrais mots"
    # i.e. sans caractères spéciaux
    for text in texts:
        words_only = re.findall(r"\w+", text)
        if (not steeming):
            [words.add(x) for x in words_only if not (x in english_stop)]
        else:
            stemmer = SnowballStemmer("english")
            [words.add(stemmer.stem(x)) for x in words_only if not (x in english_stop)]

    vocabulary = dict()
    for i, word in enumerate(words):
        vocabulary[word] = i

    n_features = len(vocabulary)
    counts = np.zeros((len(texts), n_features))

    # On construit la matrice counts en comptant pour
    # chaque mot de chaque texte son nombre d'occurences
    for i in range(len(texts)):
        words_only = re.findall(r"\w+", texts[i])
        for word in words_only:
            if (steeming):
                word = stemmer.stem(word)
            if not (word in english_stop):
                counts[i, vocabulary[word]] += 1

    return vocabulary, counts

vocabulary, X = count_words(texts)

In [4]:
###############################################################################
# Question 2
###############################################################################

# Les critiques sont à la base issues d'une page HTML, avec différents types de notation 
# (via des étoiles, une note sur 10, la désignation d'une lettre...)
# Pour chaque critique ils ont identifié un système de notion, puis si la note était bonne
# la critique a été considérée comme positive, par contre si la note a été mauvaise la critique
# a été considérée comme négative.

In [5]:
###############################################################################
# Question 3
###############################################################################
class NB(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.T = [0, 0]
        self.prior = [0, 0]
        self.total_words = [0, 0]

        
    def fit(self, X, y):
        
        # On calcule le ratio des textes postifs et négatifs
        self.prior[0] = len(texts_neg) / len(texts)
        self.prior[1] = len(texts_pos) / len(texts)
        
        # On compte le nombre de mots pour chaque classe
        # Sans oublier d'ajouter +1 par mot
        self.total_words = [0, 0]
        self.total_words[0] = np.sum(X[np.where(y == 0)]) + len(vocabulary)
        self.total_words[1] = np.sum(X[np.where(y == 1)]) + len(vocabulary)
        
        # On calcule pour chaque mot sa fréquence dans chaque classe
        self.T[0] = (np.sum(X[np.where(y == 0)], axis=0) + 1) / self.total_words[0]
        self.T[1] = (np.sum(X[np.where(y == 1)], axis=0) + 1) / self.total_words[1]
        return self
    

    def predict(self, X):
        score = np.zeros(len(X))
        proba = [0, 0]
        for i, doc in enumerate(X):
            # On calcule pour chaque classe le score du document
            # A la différence de l'algorithme proposé je conserve
            # les doublons de mots
            for c in [0, 1]:
                temp = doc * self.T[c]
                temp = temp[np.where(temp > 0)]
                proba[c] = sum(np.log(temp)) + math.log(self.prior[c])
            # On sélectionne le score le plus élévé
            if (proba[0] > proba[1]):
                score[i] = 0
            else:
                score[i] = 1
        return score
      
        
    def score(self, X, y):
        return np.mean(self.predict(X) == y)

In [6]:
nb = NB()
nb.fit(X, y)
nb.fit(X[::2], y[::2])
print("Le pourcentage de bons résultats est : {0:.2f} %".format(100*nb.score(X[1::2], y[1::2])))

Le pourcentage de bons résultats est : 82.20 %


In [56]:
###############################################################################
# Question 4
###############################################################################
# La valeur choisie pour K doit être un diviseur du nombre de sample
# Pour que le reste de l'algorithme fonctionne
K = 5

# Définition des données de training et de test
X_train_K = [0] * K
y_train_K = [0] * K
X_test_K = [0] * K
y_test_K = [0] * K

error_K = [0] * K
list_models = [0] * K


# Alimentation des données de training et de test
# à partir de X et y
for i in range(K):
    value_K = list(range(K))
    value_K.remove(i)
    
    # Alimentation des données de test
    X_test_K[i] = X[i::K]
    y_test_K[i] = y[i::K]
    
    # Alimentation des données de training
    for j in value_K:
        if (type(X_train_K[i]) == int) :
            X_train_K[i] = X[j::K]
            y_train_K[i] = y[j::K]
        else:
            X_train_K[i] = np.append(X_train_K[i], X[j::K], axis=0)
            y_train_K[i] = np.append(y_train_K[i], y[j::K], axis=0)

            
# Entrainement du modèle sur chaque données de training
# Puis calcul de l'erreur sur les données de tests respectives
for i in range(K):
    nb = NB()
    list_models[i] = nb
    nb.fit(X_train_K[i], y_train_K[i])
    error_K[i] = np.mean(np.abs(nb.predict(X_test_K[i]) - y_test_K[i]))
    print("Le pourcentage de bons résultats pour le modèle {0} est de : {1:.2f} %".format(i, 100*(1 - error_K[i])))

print('*******************************************************')
print("le pourcentage moyen de bons résultats est de : {0:.2f} %".format(100*(1 - np.mean(error_K))))
print('*******************************************************')

Le pourcentage de bons résultats pour le modèle 0 est de : 78.50 %
Le pourcentage de bons résultats pour le modèle 1 est de : 82.00 %
Le pourcentage de bons résultats pour le modèle 2 est de : 79.50 %
Le pourcentage de bons résultats pour le modèle 3 est de : 85.00 %
Le pourcentage de bons résultats pour le modèle 4 est de : 79.50 %
*******************************************************
le pourcentage moyen de bons résultats est de : 80.90 %
*******************************************************


In [58]:
###############################################################################
# Question 5
###############################################################################

# Constitution du dictionnaire des mots à ignorer
filenames_stop = sorted(glob(op.join('data', '*.stop')))
words_stop = [open(f).read() for f in filenames_stop]
words_stop = words_stop[0].split("\n")

# Génération de la matrice de fréquence des mots en prenant
# en compte les mots à ignorer
vocabulary, X = count_words(texts, words_stop)

# 
nb = NB()
nb.fit(X, y)
nb.fit(X[::2], y[::2])
print("Le pourcentage de bons résultats est : {0:.2f} %".format(100*nb.score(X[1::2], y[1::2])))

Le pourcentage de bons résultats est : 82.70 %


Le résultat s'est donc amélioré de 0.5 % par rapport à la question 3

In [60]:
###############################################################################
# scikitlearn - Question 1
###############################################################################

# Définition des libellés correspondant aux différentes méthodes
# utilisées pour afficher dans le message de résultat
method_used = [
    "Sans aucune customisation",
    "Avec la regex \w+ pour supprimer les caractères spéciaux",
    "En supprimant les stop words",
    "En utilisant les pondérations par l'inverse de la fréquence ",
    " En utilisant l'analyse 'char_wb' avec un  ngram_range de 2",
    " En utilisant l'analyse 'char_wb' avec un  ngram_range de 5",
    " En utilisant l'analyse 'char_wb' avec un  ngram_range de 10",
    " En utilisant l'analyse 'char' avec un  ngram_range de 2",
    " En utilisant l'analyse 'char' avec un  ngram_range de 5",
    " En utilisant l'analyse 'char' avec un  ngram_range de 10",
]

# Définition de la liste des pipeline, un pipeline correspondant 
# à une méthode
pipeline = [0] * len(method_used)

pipeline[0] = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB())])

pipeline[1] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer='word', token_pattern="\\w+")),
    ('classifier', MultinomialNB())])

pipeline[2] = Pipeline([
    ('vectorizer', CountVectorizer(stop_words="english")),
    ('classifier', MultinomialNB())])

pipeline[3] = Pipeline([
    ('vectorizer', CountVectorizer(stop_words="english")),
    ('tf_idf', TfidfTransformer()),
    ('classifier', MultinomialNB())])

pipeline[4] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char_wb", ngram_range=(2, 2), stop_words="english")),
    ('classifier', MultinomialNB())])

pipeline[5] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char_wb", ngram_range=(5, 5), stop_words="english")),
    ('classifier', MultinomialNB())])

pipeline[6] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char_wb", ngram_range=(10, 10), stop_words="english")),
    ('classifier', MultinomialNB())])

pipeline[7] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char", ngram_range=(2, 2), stop_words="english")),
    ('classifier', MultinomialNB())])

pipeline[8] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char", ngram_range=(5, 5), stop_words="english")),
    ('classifier', MultinomialNB())])

pipeline[9] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char", ngram_range=(10, 10), stop_words="english")),
    ('classifier', MultinomialNB())])

# Calcul du résultat pour chaque Pipeline
for i in range(len(pipeline)):
    pipeline[i].fit(texts[::2], y[::2])
    y_predi = pipeline[i].predict(texts[1::2])
    print(""" Le résulat pour la métode "{0}" est de : {1:.2f} %""".format(method_used[i], 100*np.mean(y_predi == y[1::2])))


 Le résulat pour la métode "Sans aucune customisation" est de : 81.30 %
 Le résulat pour la métode "Avec la regex \w+ pour supprimer les caractères spéciaux" est de : 81.00 %
 Le résulat pour la métode "En supprimant les stop words" est de : 80.70 %
 Le résulat pour la métode "En utilisant les pondérations par l'inverse de la fréquence " est de : 80.00 %
 Le résulat pour la métode " En utilisant l'analyse 'char_wb' avec un  ngram_range de 2" est de : 68.10 %
 Le résulat pour la métode " En utilisant l'analyse 'char_wb' avec un  ngram_range de 5" est de : 82.40 %
 Le résulat pour la métode " En utilisant l'analyse 'char_wb' avec un  ngram_range de 10" est de : 82.60 %
 Le résulat pour la métode " En utilisant l'analyse 'char' avec un  ngram_range de 2" est de : 68.10 %
 Le résulat pour la métode " En utilisant l'analyse 'char' avec un  ngram_range de 5" est de : 83.10 %
 Le résulat pour la métode " En utilisant l'analyse 'char' avec un  ngram_range de 10" est de : 84.20 %


On constate que la méthode de base a un score légèrement inférieur à celui obtenu par programmation manuelle. Cela peut venir soit du fait que le dictionnaire produit est légèrement différent, soit du fait que les mots en doublons dans un texte à analyser ne sont compter qu'une fois

On obtient un score meilleur en utilisant l'option 'char_wb' avec des groupes de lettres faisant au moins 5 caractères. Cette façon de faire est probablement meilleure car cela permet d'agréger des mots ayant des racines communes.

L'option 'char' donne un bon score avec des groupes de lettres faisant de 5 à 10 caractères, car cette option permet au final de prendre en compte l'ordonnancement des mots qui peut avoir un impact sur le sens du texte

In [61]:
###############################################################################
# scikitlearn - Question 2
###############################################################################

# Nous répétons la même méthodologie que précédemment
# mais avec comme classifier la régression logistique

pipeline[0] = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression())])

pipeline[1] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer='word', token_pattern="\\w+")),
    ('classifier', LogisticRegression())])

pipeline[2] = Pipeline([
    ('vectorizer', CountVectorizer(stop_words="english")),
    ('classifier', LogisticRegression())])

pipeline[3] = Pipeline([
    ('vectorizer', CountVectorizer(stop_words="english")),
    ('tf_idf', TfidfTransformer()),
    ('classifier', LogisticRegression())])

pipeline[4] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char_wb", ngram_range=(2, 2), stop_words="english")),
    ('classifier', LogisticRegression())])

pipeline[5] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char_wb", ngram_range=(5, 5), stop_words="english")),
    ('classifier', LogisticRegression())])

pipeline[6] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char_wb", ngram_range=(10, 10), stop_words="english")),
    ('classifier', LogisticRegression())])

pipeline[7] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char", ngram_range=(2, 2), stop_words="english")),
    ('classifier', LogisticRegression())])

pipeline[7] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char", ngram_range=(5, 5), stop_words="english")),
    ('classifier', LogisticRegression())])

pipeline[8] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char", ngram_range=(10, 10), stop_words="english")),
    ('classifier', LogisticRegression())])

for i in range(len(pipeline)):
    pipeline[i].fit(texts[::2], y[::2])
    y_predi = pipeline[i].predict(texts[1::2])
    print(""" Le résulat pour la métode "{0}" est de : {1:.2f} %""".format(method_used[i], np.mean(y_predi == y[1::2])))

 Le résulat pour la métode "Sans aucune customisation" est de : 0.83 %
 Le résulat pour la métode "Avec la regex \w+ pour supprimer les caractères spéciaux" est de : 0.84 %
 Le résulat pour la métode "En supprimant les stop words" est de : 0.83 %
 Le résulat pour la métode "En utilisant les pondérations par l'inverse de la fréquence " est de : 0.80 %
 Le résulat pour la métode " En utilisant l'analyse 'char_wb' avec un  ngram_range de 2" est de : 0.66 %
 Le résulat pour la métode " En utilisant l'analyse 'char_wb' avec un  ngram_range de 5" est de : 0.84 %
 Le résulat pour la métode " En utilisant l'analyse 'char_wb' avec un  ngram_range de 10" est de : 0.84 %
 Le résulat pour la métode " En utilisant l'analyse 'char' avec un  ngram_range de 2" est de : 0.82 %
 Le résulat pour la métode " En utilisant l'analyse 'char' avec un  ngram_range de 5" est de : 0.84 %
 Le résulat pour la métode " En utilisant l'analyse 'char' avec un  ngram_range de 10" est de : 0.84 %


En règle général on constate que le classifieur de régression logistique donne de meilleur résulats que le classifieur Naive Bayes pour ce cas de classification (il n'y a que le dernier résultat qui est légèrement moins bon

In [62]:
# Regression linear SVC
pipeline[0] = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LinearSVC())])

pipeline[1] = Pipeline([
    ('vectorizer', CountVectorizer(stop_words="english")),
    ('classifier', LinearSVC())])

pipeline[2] = Pipeline([
    ('vectorizer', CountVectorizer(stop_words="english")),
    ('tf_idf', TfidfTransformer()),
    ('classifier', LinearSVC())])

pipeline[3] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char_wb", ngram_range=(2, 2), stop_words="english")),
    ('classifier', LinearSVC())])

pipeline[4] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char_wb", ngram_range=(5, 5), stop_words="english")),
    ('classifier', LinearSVC())])

pipeline[5] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char_wb", ngram_range=(10, 10), stop_words="english")),
    ('classifier', LinearSVC())])

pipeline[6] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char", ngram_range=(2, 2), stop_words="english")),
    ('classifier', LinearSVC())])

pipeline[7] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char", ngram_range=(5, 5), stop_words="english")),
    ('classifier', LinearSVC())])

pipeline[8] = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="char", ngram_range=(10, 10), stop_words="english")),
    ('classifier', LinearSVC())])

for i in range(len(pipeline)):
    pipeline[i].fit(texts[::2], y[::2])
    y_predi = pipeline[i].predict(texts[1::2])
    print(""" Le résulat pour la métode "{0}" est de : {1:.2f} %""".format(method_used[i], np.mean(y_predi == y[1::2])))

 Le résulat pour la métode "Sans aucune customisation" est de : 0.81 %
 Le résulat pour la métode "Avec la regex \w+ pour supprimer les caractères spéciaux" est de : 0.82 %
 Le résulat pour la métode "En supprimant les stop words" est de : 0.83 %
 Le résulat pour la métode "En utilisant les pondérations par l'inverse de la fréquence " est de : 0.65 %
 Le résulat pour la métode " En utilisant l'analyse 'char_wb' avec un  ngram_range de 2" est de : 0.83 %
 Le résulat pour la métode " En utilisant l'analyse 'char_wb' avec un  ngram_range de 5" est de : 0.83 %
 Le résulat pour la métode " En utilisant l'analyse 'char_wb' avec un  ngram_range de 10" est de : 0.66 %
 Le résulat pour la métode " En utilisant l'analyse 'char' avec un  ngram_range de 2" est de : 0.83 %
 Le résulat pour la métode " En utilisant l'analyse 'char' avec un  ngram_range de 5" est de : 0.84 %
 Le résulat pour la métode " En utilisant l'analyse 'char' avec un  ngram_range de 10" est de : 0.84 %


En règle général on constate que le classifieur LinearSVC donne à peu près les mêmes résultats que le classifieur Naive Bayes pour ce cas là

In [46]:
###############################################################################
# scikitlearn - Question 3
###############################################################################

# Calcul de la matrice X des fréquences des mots en
# activant la fonctionnalité de stemming
vocabulary, X = count_words(texts, stemming=True)
nb = NB()
nb.fit(X, y)
nb.fit(X[::2], y[::2])
print("Le pourcentage de bons résultats avec racinisation est de : {}".format(nb.score(X[1::2], y[1::2])))

# On convertit le dictionnaire des stop en ne conservant que les racines
stemmer = SnowballStemmer("english")
words_stop_stemmer = set(map(lambda x : stemmer.stem(x), words_stop))

vocabulary, X = count_words(texts, english_stop=list(words_stop_stemmer), stemming=True)
nb = NB()
nb.fit(X, y)
nb.fit(X[::2], y[::2])
print("Le pourcentage de bons résultats avec words_stop et racinisation est de : {0:.2f} %".format(nb.score(X[1::2], y[1::2])))

Le pourcentage de bons résultats avec racinisation est de : 0.827
Le pourcentage de bons résultats avec words_stop et racinisation est de : 0.829


On constate que la racinisation des mots améliore quelque peu le résultat:
- +0.5% pour le cas nominal
- +0.2% pour le cas en prenant en compte les stop words

Cela semble logique puisque les mots de même racine vont être regroupés en un seul mot, et ainsi le poids de ces derniers ne seront pas dispersés en fonction de l'orthographe et de la forme des mots (exemple : "dog" et "dogs" seront regroupés dans l'unique mot "dog")

In [93]:
###############################################################################
# scikitlearn - Question 4
###############################################################################


# Fonction qui supprime de la matrice les colonnes qui
# ne sont ni des noms, verbes, adjectifs ou adverbes
def remove_types_of_words (X):
    # Calcul des indices des mots à supprimer
    index_to_delete = list()
    for word, index in vocabulary.items():
        word_token = word_tokenize(word)
        if not (pos_tag(word_token)[0][1] in ('NN', 'VBZ', 'RB','JJ')):
            index_to_delete.append(index)

    # Réduction de la matrice X initiale en supprimant
    # les colonnes issues du traitement précédent 
    X_reduce = np.delete(X, index_to_delete, axis=1)
    
    return X_reduce

    
# Calcul du résultat en supprimant les mots qui ne sont pas du
# type désiré
vocabulary, X = count_words(texts)
X_reduce = remove_types_of_words(X)
nb = NB()
nb.fit(X_reduce, y)
nb.fit(X_reduce[::2], y[::2])
print("Le pourcentage de bons résultats en supprimant des catégories de mots est de : {0:.2f} %".format(100*nb.score(X_reduce[1::2], y[1::2])))


# Calcul du résultat en supprimant les mots qui ne sont pas du
# type désiré et que ne sont pas des stop words
vocabulary, X = count_words(texts, english_stop=words_stop)
X_reduce = remove_types_of_words(X)
nb = NB()
nb.fit(X_reduce, y)
nb.fit(X_reduce[::2], y[::2])
print("Le pourcentage de bons résultats avec words_stop et en supprimant des catégories de mots est de : {0:.2f} %".format(100*nb.score(X_reduce[1::2], y[1::2])))

Le pourcentage de bons résultats en supprimant des catégories de mots est de : 82.30 %
Le pourcentage de bons résultats avec words_stop et en supprimant des catégories de mots est de : 81.70 %


En supprimant les mots qui ne sont ni des verbes, ni des noms communs, ni des adjectifs, ni des adverbes on améliore le score de 0.1%
On n'aurait pu pensé que le score aurait été un peu plus amélioré.
De plus lorsque l'on supprime les stop words le score est dégradé. Certains des stop words doivent donc en fait avoir une signification utile pour le modèle.